In [1]:
# !py -3 -m pip install pandas
# !py -3 -m pip install teradatasql
# !py -3 -m pip install teradatasqlalchemy

In [2]:
import pandas as pd
# import teradatasql
import sqlalchemy
import os

In [3]:
user_name = 'KHU9683'
pword = 'Mer82@dme'
host_name='EDWPROD.DW.MEDCITY.NET'

In [4]:
td_engine = sqlalchemy.create_engine('teradatasql://' + host_name + '/?user=' + user_name + '&password=' + pword + '&logmech=LDAP')

In [5]:
#path of file with table name and database name in csv file 
input_path = "InputFiles\\tables_duplicate_checks.csv"
output_folder = "OutputFiles\\Duplicate_Checks\\"

In [6]:
#Write file to local directory
def write_file_local(path,file_data):
    
    with open(path, 'w') as file:
        file_string = '\n'.join(file_data)
        file.write(file_string)

In [7]:
#get the DDLs for the tables from teradata 
def gen_duplicate_check():
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
# with teradatasql.connect(host=host_name, user=user_name, password=pword, logmech="LDAP") as connect:
    df = pd.read_csv(input_path, index_col=None)        

    for index, row in df.iterrows():
        try:
            database_name = str(row['Database']).strip().lower()
            table_name = str(row['Table']).strip().lower()

            duplicate_check_base_code_begin = []
            duplicate_check_base_code_begin.append("/* Test Unique Index constraint set in Teradata */")
            duplicate_check_base_code_begin.append("SET DUP_COUNT = (")
            duplicate_check_base_code_begin.append(chr(9) + "SELECT COUNT(*)")
            duplicate_check_base_code_begin.append(chr(9) + "FROM (")
            duplicate_check_base_code_begin.append(chr(9) + "SELECT")
            duplicate_check_base_code_middle = []
            duplicate_check_base_code_middle.append(chr(9) + "HAVING COUNT(*) > 1")
            duplicate_check_base_code_middle.append(chr(9) + ")")
            duplicate_check_base_code_middle.append(");")
            duplicate_check_base_code_middle.append("IF DUP_COUNT <> 0 THEN")
            duplicate_check_base_code_middle.append(chr(9) + "ROLLBACK TRANSACTION;")
            duplicate_check_base_code_end = []
            duplicate_check_base_code_end.append("ELSE")
            duplicate_check_base_code_end.append(chr(9) + "COMMIT TRANSACTION;")
            duplicate_check_base_code_end.append("END IF;")

            query = "Select TRIM(TRAILING ',' FROM (XMLAGG( TRIM(columnName) || ',' ORDER BY columnposition) (VARCHAR(1000)))) uniq_col_list "\
                "From dbc.indicesV WHERE LOWER(DatabaseName) = '{}' AND LOWER(TableName) = '{}'"\
                " AND UniqueFlag = 'Y';".format(database_name, table_name)
            results_df = pd.read_sql(query, td_engine)

            if database_name == "edwhr":
                param_name = "param_hr_core_dataset_name"
            elif database_name == "edwpi":
                param_name = "param_pi_core_dataset_name"

            for index, results_df_rec in results_df.iterrows():
                duplicate_check_code = []
                duplicate_check_code.extend(duplicate_check_base_code_begin)
                duplicate_check_code.append(chr(9) + results_df_rec['uniq_col_list'])
                duplicate_check_code.append(chr(9) + "from {{ params." + param_name + " }}." + table_name)
                duplicate_check_code.append(chr(9) + "Group BY " + results_df_rec['uniq_col_list'])
                duplicate_check_code.extend(duplicate_check_base_code_middle)
                duplicate_check_code.append(chr(9) + "RAISE USING MESSAGE = CONCAT('Duplicates are not allowed in the table: {{ params." + param_name + " }}." + table_name + "');")
                duplicate_check_code.extend(duplicate_check_base_code_end)
                output_path = output_folder + ("{}.sql".format(table_name))
                write_file_local(output_path, duplicate_check_code)

        except Exception as e1:
            print(e1)
            print("Database : {}, Table : {}".format(database_name,table_name))
            pass

    print("Tables for Record Counts : ", len(df))

In [8]:
print("Begin of Processing")

gen_duplicate_check()

print("End of Processing")

Begin of Processing


Tables for Record Counts :  1
End of Processing
